In [144]:
import pandas as pd

In [145]:
df = pd.read_csv("nba_games.csv", index_col=0)

In [146]:
df = df.sort_values("date")

In [147]:
df = df.reset_index(drop=True)

In [148]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [149]:
def add_target(group):
    group["target"] = group["won"].shift(-1)
    return group

df = df.groupby("team", group_keys=False).apply(add_target).reset_index(drop=True)
df = df.copy()


/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_9102/3664484873.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group["target"] = group["won"].shift(-1)
/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_9102/3664484873.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group["target"] = group["won"].shift(-1)
/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_9102/3664484873.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

In [150]:
df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors="ignore")

/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_9102/2793819745.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_9102/27938197

In [151]:
df["won"].value_counts()

won
False    8886
True     8886
Name: count, dtype: int64

In [152]:
df["target"].value_counts()

target
1    8872
0    8870
2      30
Name: count, dtype: int64

In [153]:
nulls = pd.isnull(df).sum()

In [154]:
nulls = nulls[nulls > 0]

In [155]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [156]:
df = df[valid_columns].copy()

In [157]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline

logistic_model = LogisticRegression(max_iter=1000,
                                    C=0.1,
                                    penalty='l2',
                                    solver='lbfgs'
                                    )

split = TimeSeriesSplit(n_splits=3)

selector = SequentialFeatureSelector(logistic_model, 
                                n_features_to_select=30, 
                                direction="forward",
                                cv=split,
                                n_jobs=-1
                               )

#Najlepsze parametry:  {'classification__C': 0.1, 'classification__penalty': 'l2', 'classification__solver': 'lbfgs'}

In [158]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [159]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [160]:
selector.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=LogisticRegression(C=0.1, max_iter=1000),
                          n_features_to_select=30, n_jobs=-1)

In [161]:
predictors = list(selected_columns[selector.get_support()])

In [162]:
predictors

['mp',
 'pf',
 'usg%',
 'ft_max',
 'trb_max',
 'tov_max',
 '+/-_max',
 'drb%_max',
 'trb%_max',
 'stl%_max',
 'tov%_max',
 'usg%_max',
 'fg%_opp',
 '3p_opp',
 'blk_opp',
 'efg%_opp',
 'blk%_opp',
 'usg%_opp',
 'fga_max_opp',
 '3p%_max_opp',
 'ft_max_opp',
 'fta_max_opp',
 'ast_max_opp',
 'blk_max_opp',
 'pts_max_opp',
 'drb%_max_opp',
 'ast%_max_opp',
 'stl%_max_opp',
 'blk%_max_opp',
 'usg%_max_opp']

In [163]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [164]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

def find_team_averages(team):
    numeric_cols = team.select_dtypes(include="number")
    rolling = numeric_cols.rolling(10).mean()
    
    rolling[["team", "season"]] = team[["team", "season"]]
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_9102/410169298.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


In [165]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)

In [166]:
df = df.dropna()

In [167]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_9102/1688004556.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_9102/1688004556.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x,

In [168]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]], left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])

In [169]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,SAC,TOR,TOR,SAC,2015-11-15
1,TOR,SAC,SAC,TOR,2015-11-15
2,CLE,DET,DET,CLE,2015-11-17
3,GSW,TOR,TOR,GSW,2015-11-17
4,DEN,NOP,NOP,DEN,2015-11-17
...,...,...,...,...,...
15769,BOS,GSW,GSW,BOS,2022-06-10
15770,GSW,BOS,BOS,GSW,2022-06-13
15771,BOS,GSW,GSW,BOS,2022-06-13
15772,GSW,BOS,BOS,GSW,2022-06-16


In [170]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [171]:
removed_columns

['team_x',
 'team_opp',
 'date',
 'team_10_x',
 'team_opp_next_x',
 'date_next',
 'team_10_y',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [178]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]
selector.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=LogisticRegression(C=0.1, max_iter=1000),
                          n_features_to_select=30, n_jobs=-1)

In [179]:
predictors = list(selected_columns[selector.get_support()])

In [180]:
predictions = backtest(full, logistic_model, predictors)

In [181]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions["actual"], predictions["prediction"])

0.6200774984229972

In [182]:
wynik = accuracy_score(predictions["actual"], predictions["prediction"])

In [183]:
print(f"wynik to {wynik*100:.4f}"+"% dokładności osiągnięte przez najlepszy model Logistycznej Regresji z selektorem Sequential Feature Selector")

wynik to 62.0077% dokładności osiągnięte przez najlepszy model Logistycznej Regresji z selectorem Sequential Feature Selector
